In [2]:
from theano import tensor as T
import lasagne as nn
import numpy as np
import pickle
from input_process import read_images

In [4]:
size = (116, 116)
pkl_file = open('../fnames_collection_2', 'r')
fnames = pkl_file.readline()
fnames = fnames.split(' ')
for i in range (0,len(fnames)):
	fnames[i] = str(fnames[i])
	fnames[i] = 'image' + fnames[i] + '.png'

train_index = fnames[0:4]

label_path = '../NewPNGlabeled/'
ori_path = '../JpegOriginalImg/'
train_input, train_label = read_images(label_path, ori_path, train_index, size)
#test_input, test_label, shape = read_images(label_path, ori_path, test_index, size)

In [17]:
#input_var = T.tensor4('input_var') 
#label_var = T.tensor4('label_var') 
input_var = train_input
label_var = train_label
shape = [4,1,116,116]
layer = nn.layers.InputLayer(shape,input_var)

In [18]:
layer = nn.layers.Conv2DLayer(layer, num_filters = 4,filter_size = 3,
                              nonlinearity = nn.nonlinearities.sigmoid, pad = 'same') #116 
layer.W.eval()

array([[[[-0.13907561, -0.16274703, -0.246071  ],
         [ 0.34266898, -0.12821651,  0.22992317],
         [-0.34619913,  0.03689676, -0.04959954]]],


       [[[-0.20934623, -0.28061303,  0.32375282],
         [ 0.23259209, -0.24612737, -0.12392155],
         [-0.10730103,  0.15955499, -0.27652889]]],


       [[[ 0.0106604 , -0.30714706, -0.20883048],
         [ 0.02040543, -0.23407865,  0.28937557],
         [-0.22165318,  0.081631  ,  0.0418348 ]]],


       [[[-0.02686872, -0.27733904, -0.16258191],
         [-0.28696147,  0.33314383, -0.27885401],
         [-0.26832548,  0.04937676,  0.32599771]]]], dtype=float32)

In [19]:
layer.b.eval()

array([ 0.,  0.,  0.,  0.], dtype=float32)

In [20]:
layer = nn.layers.Conv2DLayer(layer, num_filters = 1,filter_size = 3,
                               nonlinearity = nn.nonlinearities.sigmoid, pad = 'same')
layer.W.eval()

array([[[[ 0.21527916,  0.29668117,  0.25171503],
         [ 0.32776523,  0.01358866, -0.25231129],
         [-0.34097081,  0.24790588, -0.0305054 ]],

        [[ 0.31623673, -0.33522487, -0.20176767],
         [-0.10885017,  0.08209688, -0.16428117],
         [-0.29994527,  0.33743641,  0.09881429]],

        [[ 0.25733709,  0.13877778, -0.31574667],
         [-0.33364958, -0.08675556,  0.16213627],
         [-0.35095093, -0.28174701, -0.23183024]],

        [[-0.3605375 ,  0.02346376,  0.11612003],
         [ 0.14563733, -0.26903889, -0.13884486],
         [-0.1927785 , -0.2127572 ,  0.07561583]]]], dtype=float32)

In [21]:
layer.b.eval()

array([ 0.], dtype=float32)

In [22]:
output = nn.layers.get_output(layer) # parameters will be updated - use for train
#output_det = nn.layers.get_output(layer, deterministic=True) 

params = nn.layers.get_all_params(layer)
loss = -2*T.sum(output*label_var)/T.sum(output+label_var+0.0001)
#print (loss.eval())

lr = 1 # learning rate
updates = nn.updates.adam(loss, params, learning_rate=lr)

In [23]:
params[1]

b

In [27]:
updates[params[2]].eval()

array([[[[ 1.2152493 ,  1.29665041,  1.25168502],
         [ 1.32773459,  1.01355684,  0.74765795],
         [ 0.6589995 ,  1.24787533,  0.96946472]],

        [[ 1.31620693,  0.66474438,  0.79820234],
         [ 0.89111918,  1.08206522,  0.83568794],
         [ 0.7000252 ,  1.3374058 ,  1.09878445]],

        [[ 1.25730729,  1.13874698,  0.68422347],
         [ 0.66631985,  0.9132129 ,  1.16210568],
         [ 0.6490196 ,  0.7182225 ,  0.76814008]],

        [[ 0.63943255,  1.02343285,  1.11609018],
         [ 1.14560676,  0.73092949,  0.86112458],
         [ 0.80719197,  0.78721249,  1.07558632]]]], dtype=float32)

In [ ]:
w_1 = np.random.normal(0, 0.01, (3, 3))
w_2 = np.random.normal(0, 0.02, (3, 3))
a_1 = w_1.reshape(1,-1)
a_2 = w_2.reshape(1,-1)
ve = np.concatenate((a_1,a_2),axis=1)
w_init = ve.reshape(2,1,3,3)

In [ ]:
w_init


In [ ]:
pkl_file = open('/Users/apple/Documents/Lab_Winter/GDCNN/scripts/params_epoch_400', 'rb')
params = pickle.load(pkl_file)
params[4][1]

In [ ]:
w1 = np.ones([3,3])
w1 = w1.reshape(-1,3,3)
w2 = np.ones([3,3])*2
w2 = w2.reshape(-1,3,3)
W_filter1 = np.concatenate((w1,w2),axis=0)

w1 = np.ones([3,3])
w1 = w1.reshape(-1,3,3)
w2 = np.ones([3,3])*2
w2 = w2.reshape(-1,3,3)
W_filter2 = np.concatenate((w1,w2),axis=0)

w1 = np.ones([3,3])
w1 = w1.reshape(-1,3,3)
w2 = np.ones([3,3])*2
w2 = w2.reshape(-1,3,3)
W_filter3 = np.concatenate((w1,w2),axis=0)

Ws = np.concatenate((W_filter1.reshape(-1,2,3,3),W_filter2.reshape(-1,2,3,3),W_filter3.reshape(-1,2,3,3)),axis=0)



In [ ]:
# if we want to initial weight of [2,2,3,3]
shape = [2,2,3,3]
[NumChannel,NumFilter,size,size] = shape

Ws = np.array([]).reshape(1,-1)
for i in range (0,NumFilter*NumFilter):
    w = np.ones([3,3]).reshape(1,-1) # np.ones([3,3]) can be replaced by other functions
    Ws = np.concatenate((Ws,w),axis=1)
Ws = Ws.reshape(2,2,3,3)
Ws

In [ ]:
shape = [2,2,3,3]
[NumChannel,NumFilter,size,size] = shape
NumChannel

In [ ]:
import theano
from theano import tensor as T
from theano import pp
x = T.dscalar('x')
y = T.dscalar('y')
z = x ** 2 + y ** 2
gz = T.grad(z,x)
f = theano.function([x,y],gz)
pp(f.maker.fgraph.outputs[0])

#f = theano.function([x,y],gz)
#pp(f.maker.fgraph.outputs[1])



In [ ]:
import numpy as np
nx, ny = (3, 2)
x = np.linspace(-1, 1, 3)
y = np.linspace(-1, 1, 3)
xv, yv = np.meshgrid(x, y)
xv.reshape(1,-1), yv.reshape(1,-1)
for i in x:
    print(i)

In [ ]:
x = np.linspace(-1, 1, 3)
y = np.linspace(-1, 1, 3)
[x,y] = list(map(lambda x:x.reshape(1,-1),np.meshgrid(x,y)))
print(math.floor(3/2))

In [ ]:
f = random.uniform(0.0001,2)
gamma = random.uniform(0.0001,1)
sigma = random.uniform(0.0001,2)
theta = random.uniform(0,2*math.pi)    
psi = random.uniform(0.0001,2)
    
bond = math.floor(size/2)
x_range = np.linspace(-bond, bond, size)
y_range = np.linspace(-bond, bond, size)

[x_range,y_range] = list(map(lambda x:x.reshape(1,-1),np.meshgrid(x_range,y_range)))
gfilter = []
for (x,y) in zip(np.ndarray.tolist(x_range)[0], np.ndarray.tolist(y_range)[0]):
    xt = x*math.cos(theta) + y*math.sin(theta)
    yt = -x*math.sin(theta) + y*math.cos(theta)
    z1 = -(xt**2 + (gamma*yt)**2)/(2*sigma**2)
    z2 = 1j*2*math.pi*f*xt+psi
    value = f**2/(math.pi*gamma)*math.exp(z1)*cmath.exp(z2)
    gfilter.append(value.real)



In [ ]:
import cmath
cmath.exp(2j)

In [ ]:
import math
from theano import tensor
import theano
from theano import pp

math.exp(0)
f = T.fscalar('f')
gamma = T.fscalar('gamma')
sigma = T.fscalar('sigma')
theta = T.fscalar('theta')
psi = T.fscalar('psi')
x = T.fscalar('x')
y = T.fscalar('y')

xt = x*T.cos(theta) + y*T.sin(theta)
yt = -x*T.sin(theta) + y*T.cos(theta)
z1 = -(xt**2 + (gamma*yt)**2)/(2*sigma**2)
z2 = 1j*2*math.pi*f*xt+psi
value = f**2/(math.pi*gamma)*math.exp(z1)*cmath.exp(z2)

gz = T.grad(value,theta)
f = theano.function([x,y,f,gamma,sigma,theta,psi],gz)
pp(f.maker.fgraph.outputs[0])

In [ ]:
from theano import tensor
import theano
from theano import pp
xt = x*tensor.cos(2*theta)
gz = theano.grad(xt,x)
#f = theano.function([x,theta],gz)
#pp(f.maker.fgraph.outputs[0])
gz_new = gz*1.5
f = theano.function([x,theta],gz_new)
f(1,1)


In [ ]:
math.cos(2)*1.5

In [42]:
index = np.ndarray.tolist(np.random.randint(0,10,5))
index[0:2]

[1, 2]

In [51]:
a = [1,2,3,4,5]
random.shuffle(a)
print(a)

[1, 4, 5, 2, 3]
